In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from tqdm import tqdm
from datetime import date, datetime
import time

In [ ]:
startyear = 2017
endyear = 2018
d0 = date(1899, 12, 30)

In [ ]:
for y in range(startyear, endyear+1):
    results = []
    if y>datetime.now().year:
        break
        
    for m in range(1, 12+1):
        if y==datetime.now().year and m>datetime.now().month:
            break
            
        if m==1 or m==3 or m==5 or m==7 or m==8 or m==10 or m==12: days = 31
        elif m==4 or m==6 or m==9 or m==11: days = 30
        else:
            if y%4==0 and (y%400==0 or y%100!=0): days = 29
            else: days = 28
        
        for d in tqdm(range(1, days+1)):
            if y==datetime.now().year and m>datetime.now().month and d>=datetime.now().day:
                break
                
            d1 = date(y, m, d)
            delta = d1 - d0
            
            time.sleep(2)
            
            page_link = 'https://timesofindia.indiatimes.com/{}/{}/{}/archivelist/year-{},month-{},starttime-{}.cms'.format(y, m, d, y, m, delta.days)
            page_response = requests.get(page_link)
            page_content = BeautifulSoup(page_response.content, "html.parser")
            
            for link in page_content.find_all('a', href=True):
                if 'http://timesofindia.indiatimes.com//' in link['href']:
                    article = []
                    article.append(m)
                    article.append(y)
                    article.append(link.text)
                    article.append(link['href'])
                    if 'HIV' in link.text:
                        results.append(article)
                        
    alpha = pd.DataFrame(results, columns=['month', 'year', 'name', 'link'])
    file_name = str(y) + '.csv'
    alpha.to_csv(file_name)
    print(file_name)

In [ ]:
for y in range(startyear, endyear+1):
    link_file = str(y) + '.csv'
    alpha = pd.read_csv(link_file, index_col=0)
    results = []
    
    for link in tqdm(range(len(alpha))):

        time.sleep(2)

        page_link = alpha.link[link]
        page_response = requests.get(page_link)
        page_content = BeautifulSoup(page_response.content, "html.parser")
        string = page_content.find_all('div', attrs={'class':'Normal'})

        if len(string)!=0:

            string = re.sub('<[^>]+>', '', str(string[0]))
            string = re.sub('\n', '', str(string))
            string = re.sub(' +', ' ', str(string))

            month = str(alpha.month[link])
            year = str(alpha.year[link])

            article = []

            article.append(month)
            article.append(year)
            article.append(alpha.name[link])
            article.append(alpha.link[link])
            article.append(string)

            results.append(article)

    alpha = pd.DataFrame(results, columns=['month', 'year', 'name', 'link', 'article'])
    file_name = str(y) + '_articles.csv'
    alpha.to_csv(file_name)

In [ ]:
alpha